In [6]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import xgboost as xgb
import pandas as pd
from sklearn.metrics import accuracy_score

# Load datasets
team_stats_df = pd.read_csv('team_stats_0423_sorted.csv')
standings_df = pd.read_csv('NBA_Standings_Ranked_Classes.csv')
elo_df = pd.read_csv('elo_ratings_per_year.csv')
four_factors_df = pd.read_csv('team_year_avg_four_factors.csv')

# Merge the Elo ratings to stats dataset
team_stats_df = pd.merge(team_stats_df, elo_df[['Year', 'Team', 'Elo Rating']], how='left', on=['Year', 'Team'])

# Merge the ff to  team stats dataset
team_stats_df = pd.merge(team_stats_df, four_factors_df[['Year', 'Team', 'eFG%', 'TOV%', 'ORB%', 'FT_Rate']], how='left', on=['Year', 'Team'])

# Merge the team stats with the updated standings data
merged_data = pd.merge(team_stats_df, standings_df, how='left', on=['Year', 'Team'])

# Filter the data by year range
filtered_data = merged_data[merged_data['Year'] >= 2004]

# Select the relevant features for training
features = ['Elo Rating', 'eFG%', 'TOV%', 'ORB%', 'FT_Rate']



C:\Users\janmi\AppData\Local\Temp\ipykernel_20096\3990241616.py:14: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  team_stats_df = pd.read_csv('team_stats_0423_sorted.csv')


In [7]:
def train_and_evaluate_yearly(conference_data, start_year, end_year, model):
    # Train on data from start_year to end_year-1 and test on next year
    train_data = conference_data[(conference_data['Year'] >= start_year) & (conference_data['Year'] < end_year)]
    test_data = conference_data[conference_data['Year'] == (end_year + 1)]
    
    X_train = train_data[features]
    y_train = train_data['ranking_class']

    X_test = test_data[features]
    y_test = test_data['ranking_class']

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = model.predict(X_test)

    # Calculate and return the accuracy score
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy


In [8]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(random_state=42)


results_summary = []

# Iterate over the years using 1 and 2 years of data to predict the next year
for window_size in [1, 2]:
    for year in range(2008, 2023):
        eastern_data = filtered_data[filtered_data['Conference'] == 'Eastern Conference']
        western_data = filtered_data[filtered_data['Conference'] == 'Western Conference']

        # Train and evaluate for East
        eastern_accuracy = train_and_evaluate_yearly(eastern_data, year - window_size, year, model)
        
        # Train and evaluate for West
        western_accuracy = train_and_evaluate_yearly(western_data, year - window_size, year, model)
        
        
        results_summary.append({
            "Year": year,
            "Window_Size": window_size,
            "Model": "RandomForest",
            "Eastern_Accuracy": eastern_accuracy,
            "Western_Accuracy": western_accuracy
        })

# Convert results to DataFrame
results_df = pd.DataFrame(results_summary)

# Group by window size and average the accuracies
RF_FF = results_df.groupby(['Window_Size']).mean()[['Eastern_Accuracy', 'Western_Accuracy']]
print(RF_FF)


             Eastern_Accuracy  Western_Accuracy
Window_Size                                    
1                    0.572128          0.589136
2                    0.600149          0.590480


C:\Users\janmi\AppData\Local\Temp\ipykernel_20096\1509196490.py:34: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  RF_FF = results_df.groupby(['Window_Size']).mean()[['Eastern_Accuracy', 'Western_Accuracy']]


In [9]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000, random_state=42)

results_summary = []

# Iterate over the years using 1 and 2 years of data to predict the next year
for window_size in [1, 2]:
    for year in range(2008, 2023):
        eastern_data = filtered_data[filtered_data['Conference'] == 'Eastern Conference']
        western_data = filtered_data[filtered_data['Conference'] == 'Western Conference']

        # Train and evaluate for East
        eastern_accuracy = train_and_evaluate_yearly(eastern_data, year - window_size, year, model)
        
        # Train and evaluate for West
        western_accuracy = train_and_evaluate_yearly(western_data, year - window_size, year, model)
        
        
        results_summary.append({
            "Year": year,
            "Window_Size": window_size,
            "Model": "RandomForest",
            "Eastern_Accuracy": eastern_accuracy,
            "Western_Accuracy": western_accuracy
        })

# Convert results to a DataFrame
results_df = pd.DataFrame(results_summary)

# Group by window size and average the accuracies
LR_FF = results_df.groupby(['Window_Size']).mean()[['Eastern_Accuracy', 'Western_Accuracy']]
print(LR_FF)


C:\Users\janmi\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\janmi\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\janmi\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\janmi\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\janmi\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\janmi\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

             Eastern_Accuracy  Western_Accuracy
Window_Size                                    
1                    0.652364          0.701506
2                    0.674869          0.688419


C:\Users\janmi\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\janmi\AppData\Local\Temp\ipykernel_20096\944775101.py:32: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  LR_FF = results_df.groupby(['Window_Size']).mean()[['Eastern_Accuracy', 'Western_Accuracy']]


In [10]:
from sklearn.svm import SVC


model = SVC(random_state=42)
results_summary = []

# Iterate over the years using 1 and 2 years of data to predict the next year
for window_size in [1, 2]:
    for year in range(2008, 2023):
        eastern_data = filtered_data[filtered_data['Conference'] == 'Eastern Conference']
        western_data = filtered_data[filtered_data['Conference'] == 'Western Conference']

        # Train and evaluate for East
        eastern_accuracy = train_and_evaluate_yearly(eastern_data, year - window_size, year, model)
        
        # Train and evaluate for West
        western_accuracy = train_and_evaluate_yearly(western_data, year - window_size, year, model)
        
        results_summary.append({
            "Year": year,
            "Window_Size": window_size,
            "Model": "RandomForest",
            "Eastern_Accuracy": eastern_accuracy,
            "Western_Accuracy": western_accuracy
        })

# Convert results to a DataFrame
results_df = pd.DataFrame(results_summary)

# Group by window size and average the accuracies
SVC_FF = results_df.groupby(['Window_Size']).mean()[['Eastern_Accuracy', 'Western_Accuracy']]
print(SVC_FF)

             Eastern_Accuracy  Western_Accuracy
Window_Size                                    
1                    0.675169          0.648888
2                    0.679826          0.671247


C:\Users\janmi\AppData\Local\Temp\ipykernel_20096\29960641.py:31: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  SVC_FF = results_df.groupby(['Window_Size']).mean()[['Eastern_Accuracy', 'Western_Accuracy']]


In [11]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(random_state=42)
results_summary = []

# Iterate over the years using 1 and 2 years of data to predict the next year
for window_size in [1, 2]:
    for year in range(2008, 2023):
        eastern_data = filtered_data[filtered_data['Conference'] == 'Eastern Conference']
        western_data = filtered_data[filtered_data['Conference'] == 'Western Conference']

        # Train and evaluate for East
        eastern_accuracy = train_and_evaluate_yearly(eastern_data, year - window_size, year, model)
        
        # Train and evaluate for West
        western_accuracy = train_and_evaluate_yearly(western_data, year - window_size, year, model)
        
        # Store results
        results_summary.append({
            "Year": year,
            "Window_Size": window_size,
            "Model": "RandomForest",
            "Eastern_Accuracy": eastern_accuracy,
            "Western_Accuracy": western_accuracy
        })

# Convert results to DataFram
results_df = pd.DataFrame(results_summary)

# Group by window size and average the accuracies
GB_FF = results_df.groupby(['Window_Size']).mean()[['Eastern_Accuracy', 'Western_Accuracy']]
print(GB_FF)


             Eastern_Accuracy  Western_Accuracy
Window_Size                                    
1                    0.523452          0.563619
2                    0.627098          0.634597


C:\Users\janmi\AppData\Local\Temp\ipykernel_20096\2614244928.py:31: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  GB_FF = results_df.groupby(['Window_Size']).mean()[['Eastern_Accuracy', 'Western_Accuracy']]


In [12]:
from sklearn.neighbors import KNeighborsClassifier


model = KNeighborsClassifier()
results_summary = []

# Iterate over the years using 1 and 2 years of data to predict the next year
for window_size in [1, 2]:
    for year in range(2008, 2023):
        eastern_data = filtered_data[filtered_data['Conference'] == 'Eastern Conference']
        western_data = filtered_data[filtered_data['Conference'] == 'Western Conference']

        # Train and evaluate for Eastern Conference
        eastern_accuracy = train_and_evaluate_yearly(eastern_data, year - window_size, year, model)
        
        # Train and evaluate for Western Conference
        western_accuracy = train_and_evaluate_yearly(western_data, year - window_size, year, model)
        
        # Store results
        results_summary.append({
            "Year": year,
            "Window_Size": window_size,
            "Model": "RandomForest",
            "Eastern_Accuracy": eastern_accuracy,
            "Western_Accuracy": western_accuracy
        })

# Convert results to DataFrame
results_df = pd.DataFrame(results_summary)

# Group by window size and average the accuracies
KNN_FF = results_df.groupby(['Window_Size']).mean()[['Eastern_Accuracy', 'Western_Accuracy']]
print(KNN_FF)


             Eastern_Accuracy  Western_Accuracy
Window_Size                                    
1                    0.609415          0.643429
2                    0.649036          0.634955


C:\Users\janmi\AppData\Local\Temp\ipykernel_20096\1772040376.py:32: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  KNN_FF = results_df.groupby(['Window_Size']).mean()[['Eastern_Accuracy', 'Western_Accuracy']]


In [13]:
from sklearn.tree import DecisionTreeClassifier


model = DecisionTreeClassifier(random_state=42)
results_summary = []

# Iterate over the years using 1 and 2 years of data to predict the next year
for window_size in [1, 2]:
    for year in range(2008, 2023):
        eastern_data = filtered_data[filtered_data['Conference'] == 'Eastern Conference']
        western_data = filtered_data[filtered_data['Conference'] == 'Western Conference']

        # Train and evaluate for East
        eastern_accuracy = train_and_evaluate_yearly(eastern_data, year - window_size, year, model)
        
        # Train and evaluate for West
        western_accuracy = train_and_evaluate_yearly(western_data, year - window_size, year, model)
        
        # Store results
        results_summary.append({
            "Year": year,
            "Window_Size": window_size,
            "Model": "RandomForest",
            "Eastern_Accuracy": eastern_accuracy,
            "Western_Accuracy": western_accuracy
        })

# Convert results to DataFrame
results_df = pd.DataFrame(results_summary)

# Group by window size and average the accuracies
DT_FF = results_df.groupby(['Window_Size']).mean()[['Eastern_Accuracy', 'Western_Accuracy']]
print(DT_FF)

             Eastern_Accuracy  Western_Accuracy
Window_Size                                    
1                    0.573251          0.624894
2                    0.622791          0.639265


C:\Users\janmi\AppData\Local\Temp\ipykernel_20096\3912565727.py:32: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  DT_FF = results_df.groupby(['Window_Size']).mean()[['Eastern_Accuracy', 'Western_Accuracy']]


In [15]:
print('LR', LR_FF)
print('RF', RF_FF)
print('SVC', SVC_FF)
print('GB', GB_FF)
print('KNN', KNN_FF)
print('DT', DT_FF)



LR              Eastern_Accuracy  Western_Accuracy
Window_Size                                    
1                    0.652364          0.701506
2                    0.674869          0.688419
RF              Eastern_Accuracy  Western_Accuracy
Window_Size                                    
1                    0.572128          0.589136
2                    0.600149          0.590480
SVC              Eastern_Accuracy  Western_Accuracy
Window_Size                                    
1                    0.675169          0.648888
2                    0.679826          0.671247
GB              Eastern_Accuracy  Western_Accuracy
Window_Size                                    
1                    0.523452          0.563619
2                    0.627098          0.634597
KNN              Eastern_Accuracy  Western_Accuracy
Window_Size                                    
1                    0.609415          0.643429
2                    0.649036          0.634955
DT              Eastern

In [16]:
LR_FF['Combined_Accuracy'] = LR_FF.mean(axis=1)
RF_FF['Combined_Accuracy'] = RF_FF.mean(axis=1)
SVC_FF['Combined_Accuracy'] = SVC_FF.mean(axis=1)
GB_FF['Combined_Accuracy'] = GB_FF.mean(axis=1)
KNN_FF['Combined_Accuracy'] = KNN_FF.mean(axis=1)
DT_FF['Combined_Accuracy'] = DT_FF.mean(axis=1)
# Print the result
print(LR_FF[['Combined_Accuracy']])
print(RF_FF[['Combined_Accuracy']])
print(SVC_FF[['Combined_Accuracy']])
print(GB_FF[['Combined_Accuracy']])
print(KNN_FF[['Combined_Accuracy']])
print(DT_FF[['Combined_Accuracy']])

             Combined_Accuracy
Window_Size                   
1                     0.676935
2                     0.681644
             Combined_Accuracy
Window_Size                   
1                     0.580632
2                     0.595315
             Combined_Accuracy
Window_Size                   
1                     0.662029
2                     0.675537
             Combined_Accuracy
Window_Size                   
1                     0.543536
2                     0.630847
             Combined_Accuracy
Window_Size                   
1                     0.626422
2                     0.641995
             Combined_Accuracy
Window_Size                   
1                     0.599072
2                     0.631028
